In [1]:
import torch,torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.nn import *
from torch.optim import *
from tqdm import tqdm
import wandb
import cv2

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data.head()

,Date(UTC),UnixTimeStamp,Supply,MarketCap,Price
0,7/30/2015,1438214400,7.204931e+07,0.0,0.00
1,7/31/2015,1438300800,7.208550e+07,0.0,0.00
2,8/1/2015,1438387200,7.211320e+07,0.0,0.00
3,8/2/2015,1438473600,7.214143e+07,0.0,0.00
4,8/3/2015,1438560000,7.216940e+07,0.0,0.00


In [4]:
X = data['Date(UTC)']

In [7]:
new_X = []
for x in X:
    x = x.split('/')
    x = int(f'{x[0]}{x[1]}{x[2]}')
    new_X.append(x)

In [9]:
X = new_X

In [13]:
y = torch.from_numpy(np.array(data['Price'].tolist()))

ValueError: could not convert string to float: '1,006.41'